<a href="https://colab.research.google.com/github/tortas/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_4_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [8]:
import pandas as pd

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data
!ls

--2019-04-12 16:49:03--  https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32218 (31K) [application/x-httpd-php]
Saving to: ‘crx.data’

crx.data            100%[===================>]  31.46K  --.-KB/s    in 0.03s   

2019-04-12 16:49:03 (993 KB/s) - ‘crx.data’ saved [32218/32218]

Credit+Approval  Credit+Approval.1  crx.data  sample_data


In [0]:
# Import and replace ? with Nan
df = pd.read_csv('crx.data',
                 header=None,
                 na_values="?")

In [0]:
for i in range(16):
  print(df[i].unique())

In [0]:
plus_df = df[df[15] == '+']
minus_df = df[df[15] == '-']

In [29]:
plus_df.shape, minus_df.shape

((307, 16), (383, 16))

## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" (aka [contingency tables](https://en.wikipedia.org/wiki/Contingency_table)) between them and the class label, and apply the Chi-squared test to them. [pandas.crosstab](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html) can create contingency tables, and [scipy.stats.chi2_contingency](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) can calculate the Chi-squared statistic for them.

There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end - use simple visualizations, such as boxplots or a scatter matrix (both built in to pandas), to get a feel for the overall distribution of the variables.

This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [37]:
df.corr()

,1,2,7,10,13,14
1,1.000000,0.202317,0.395751,0.185912,-0.079812,0.018553
2,0.202317,1.000000,0.298902,0.271207,-0.224242,0.123121
7,0.395751,0.298902,1.000000,0.322330,-0.077163,0.051345
10,0.185912,0.271207,0.322330,1.000000,-0.120096,0.063692
13,-0.079812,-0.224242,-0.077163,-0.120096,1.000000,0.066853
14,0.018553,0.123121,0.051345,0.063692,0.066853,1.000000


In [60]:
df.describe()

,1,2,7,10,13,14
count,678.000000,690.000000,690.000000,690.00000,677.000000,690.000000
mean,31.568171,4.758725,2.223406,2.40000,184.014771,1017.385507
std,11.957862,4.978163,3.346513,4.86294,173.806768,5210.102598
min,13.750000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,22.602500,1.000000,0.165000,0.00000,75.000000,0.000000
50%,28.460000,2.750000,1.000000,0.00000,160.000000,5.000000
75%,38.230000,7.207500,2.625000,3.00000,276.000000,395.500000
max,80.250000,28.000000,28.500000,67.00000,2000.000000,100000.000000


In [35]:
from scipy import stats

# Columns of continuous variables
cont_columns = [1, 2, 7, 10, 13, 14]

for i in cont_columns:
  print(stats.ttest_ind(plus_df[i], minus_df[i], nan_policy='omit'))

Ttest_indResult(statistic=4.2922156166315535, pvalue=2.0276637071781407e-05)
Ttest_indResult(statistic=5.52998337614816, pvalue=4.551680702308068e-08)
Ttest_indResult(statistic=8.935819983773698, pvalue=3.6710537401601785e-18)
Ttest_indResult(statistic=11.667004222431277, pvalue=7.957718568079967e-29)
Ttest_indResult(statistic=-2.6358251986645476, pvalue=0.008586135473979569)
Ttest_indResult(statistic=4.680216020964486, pvalue=3.4520256956287944e-06)


In [0]:
table1 = pd.crosstab(df[0], df[15], margins=True)

In [0]:
chi_squared, p_value, dof, expected = stats.chi2_contingency(table1)

In [0]:
import numpy as np

In [59]:
print(f"Chi-Squared: {chi_squared}")
print(f"P-value: {p_value}")
print(f"Degrees of Freedom: {dof}") 
print("Expected: \n", np.array(expected))

Chi-Squared: 0.41143508194591916
P-value: 0.981530018361231
Degrees of Freedom: 4
Expected: 
 [[ 94.15929204 115.84070796 210.        ]
 [209.84070796 258.15929204 468.        ]
 [304.         374.         678.        ]]


In [56]:
table2 = pd.crosstab(df[4], df[15], margins=True)
chi_squared, p_value, dof, expected = stats.chi2_contingency(table2)
print(f"Chi-Squared: {chi_squared}")
print(f"P-value: {p_value}")
print(f"Degrees of Freedom: {dof}") 
print("Expected: \n", np.array(expected))

Chi-Squared: 26.234074966202144
P-value: 0.0002013603203375773
Degrees of Freedom: 6
Expected: 
 [[229.90789474 289.09210526 519.        ]
 [  0.88596491   1.11403509   2.        ]
 [ 72.20614035  90.79385965 163.        ]
 [303.         381.         684.        ]]


## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

**t-tests:**
I ran 2-sample t-tests across all continuous columns which compared the "plus" dataframe with the "minus" dataframe. Something interesting I found was that none of the p-values exceeded even the 0.01 significance level which indicates that we reject the null hypothesis in each case where there is no significant difference between those with a "plus" attribute and "minus". 

**chi-tests:**
In the first chi-squared test, I tested whether the feature represented  in column 0 and its plus/minus attribute in column 15 were independent. I got a very high p-value and a very low chi-squared statistic which indicates that we accept the null hypothesis that the two features are independent. In the second chi-squared test, I compared the plus/minus attribute and the feature in column 4. In this case I calculated a high chi-squared statistic and a p-value below 0.01 which tells me that I should reject that the two features are independent.

**challenging:**
The most challenging part of this sprint challenge was managing my time and completing all the required sections before time was up.

